# 04 Embeddings | 02-CosineSimilarity

## Intro


Cosine similarity is a metric used to determine the cosine of the angle between two non-zero vectors in a multi-dimensional space. It's a measure of similarity between two vectors, with a value ranging from -1 (completely dissimilar) to 1 (completely similar). 

This metric is widely used in various domains including text analysis, recommendation systems, and machine learning. The appeal of cosine similarity lies in its effectiveness, especially in high-dimensional spaces, and its independence from vector magnitude which can be particularly useful in text analysis where the length of the documents can vary significantly.

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("SKIT_EMBEDDING_DEPLOYMENTNAME") ?? "SKIT_EMBEDDING_DEPLOYMENTNAME not found";

string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

Console.WriteLine($"OpenAI Client created...");


Expected output:

```
Installed Packages
    Azure.AI.OpenAI, 1.0.0-beta.12
    DotNetEnv, 2.5.0

OpenAI Client created...
```

### Calculating Cosine similarity

The following method provides the Cosine distance between two given vectors.

Azure and OpenAI provide various tools and platforms that leverage similarity metrics like cosine similarity for various applications. For instance, Azure Machine Learning and [Azure Cognitive Services](https://docs.microsoft.com/en-us/azure/cognitive-services/) offer functionalities that allow for text analytics and building recommendation systems where cosine similarity can be employed to measure similarity between items. Similarly, OpenAI with its [GPT-3](https://openai.com/research/gpt-3) and [DALL-E](https://openai.com/research/dall-e) models, facilitates semantic search and other tasks that can benefit from similarity measurements. These platforms often provide a broader set of similarity metrics, allowing for flexibility depending on the nature of the data and the specific requirements of the task at hand.

Another alternative is to use ```MathNet.Numerics``` package and the Distance method. This method provides a variety of distance metrics, including cosine similarity. The following code snippet shows how to use this method to calculate cosine similarity between two vectors.

```
using MathNet.Numerics;
// calculate the Cosine distance between two vectors a and b
float distance = Distance.Cosine(vector a, vector b);
```

In [ ]:
static double CalculateCosineDistance(double[] vectorA, double[] vectorB)
{
    if (vectorA.Length != vectorB.Length)
    {
        throw new ArgumentException("Vectors must be the same length");
    }

    double dotProduct = 0;
    double magnitudeA = 0;
    double magnitudeB = 0;

    for (int i = 0; i < vectorA.Length; i++)
    {
        dotProduct += vectorA[i] * vectorB[i];
        magnitudeA += Math.Pow(vectorA[i], 2);
        magnitudeB += Math.Pow(vectorB[i], 2);
    }

    double cosineSimilarity = dotProduct / (Math.Sqrt(magnitudeA) * Math.Sqrt(magnitudeB));
    double cosineDistance = 1 - cosineSimilarity;

    return cosineDistance;
}

### Step 2: Create Embeddings

To calculate the embeddings/vector you have to call the method `GetEmbeddingsAsync()` on the instance of `OpenAIClient`. The method provides multiple return values where the embedding can be found in `modelResponse.Value.Data[0]`.

The ```csv``` file located under ```assets/docs/04_Embeddings/generated_questions_solutions_10.csv``` contains the questions and answers which are used for the embedding calculation. 

We will create a list of float[] and the string answers. The float[] will be used for the cosine similarity calculation and the string answers for the output.

In [ ]:
string csvPath = Path.Combine(assetsFolder, "docs" , "04_Embeddings", "generated_questions_solutions_10.csv");
EmbeddingsOptions embeddingsOptions;

List<(float[], string,string)> rows = new List<(float[], string,string)>{};

using (var reader = new StreamReader(csvPath))
{
    while (!reader.EndOfStream)
    {
        var line = reader.ReadLine();
        var values = line.Split(',');
        // in this example we only need the question and answer
        string question = values[1];
        string answer = values[2];

        embeddingsOptions = new EmbeddingsOptions(embeddingDeploymentName,new List<string> {question});
        var embeddingsResponse = await openAIClient.GetEmbeddingsAsync(embeddingsOptions);
        rows.Add((embeddingsResponse.Value.Data[0].Embedding.ToArray(),question, answer));        
    }
}


### Step 3: Calculate Cosine similarity

In our use case, a customer asks a question and the aim is to find the best suited answer for a semantically similar question.

In [ ]:
#r "nuget: MathNet.Numerics, 5.0.0"
using MathNet.Numerics;

string customerQuestion = "could my phone battary life be improved?";
// first we need to get the embeddings for the customer question
embeddingsOptions = new EmbeddingsOptions(embeddingDeploymentName,new List<string> {customerQuestion});
var embeddingsResponse = await openAIClient.GetEmbeddingsAsync(embeddingsOptions);
float[] customerQuestionEmbeddings = embeddingsResponse.Value.Data[0].Embedding.ToArray();

// now we can calculate the cosine distance between the customer question and all the questions in the csv
List<(float, string,string)> distances = new List<(float, string,string)>();

foreach (var row in rows)
{
    float distance = Distance.Cosine(customerQuestionEmbeddings, row.Item1);
    distances.Add((distance, row.Item2,row.Item3));
}

Expected output:

```
Installed Packages
    MathNet.Numerics, 5.0.0
```

In [ ]:
// printing the output of the distances

foreach (var distance in distances)
{
    Console.WriteLine($"Distance: {distance.Item1} - Question: {distance.Item2} - Answer: {distance.Item3}");
}


### Understanding the results

```
Distance: 0.23299031 - Question: "How do I set up voicemail on my Asus Zenfone 8?" - Answer: "Refer to the user manual of your Asus Zenfone 8 or check our online tutorials for voicemail setup."
Distance: 0.18871234 - Question: "I'm not receiving calls on my Sony Xperia 5. Can you help?" - Answer: "Please refer to our online support guides or contact our support team for assistance with your Sony Xperia 5 and Platinum Plan."
Distance: 0.19163826 - Question: "Is there a way to get more storage on my Samsung Galaxy S22?" - Answer: "Please refer to our online support guides or contact our support team for assistance with your Samsung Galaxy S22 and Gold Plan."
Distance: 0.2567708 - Question: "Can I change my Business Plan midway through the month?" - Answer: "Please refer to our online support guides or contact our support team for assistance with your LG Velvet and Business Plan."
Distance: 0.23483926 - Question: "How do I reset my Nokia 8.3 to factory settings?" - Answer: "Please refer to our online support guides or contact our support team for assistance with your Nokia 8.3 and Platinum Plan."
Distance: 0.2485879 - Question: "Is Phoenix covered in my Unlimited Plan's network?" - Answer: "Please refer to our online support guides or contact our support team for assistance with your Sony Xperia 5 and Unlimited Plan."
Distance: 0.22266291 - Question: "Why is my Nokia 8.3 not recognizing my SD card?" - Answer: "Please refer to our online support guides or contact our support team for assistance with your Nokia 8.3 and Student Plan."
Distance: 0.2286194 - Question: "I lost my iPhone 13. How can I track it?" - Answer: "Please refer to our online support guides or contact our support team for assistance with your iPhone 13 and Platinum Plan."
Distance: 0.22360232 - Question: "How do I backup data on my Nokia 8.3?" - Answer: "Please refer to our online support guides or contact our support team for assistance with your Nokia 8.3 and Family Plan."
Distance: 0.14397955 - Question: "How can I extend the battery life of my OnePlus 9?" - Answer: "Please refer to our online support guides or contact our support team for assistance with your OnePlus 9 and Gold Plan."
```

The smaller the distance the more similar the customer question is to the question in the list, in our case it is the last one, we can see that the question is very similar to the question in the list.

## Next step

The next steps would be to examine other methods for search. Note that the above in-memory search could be useful in many cases, however if your dataset is big and getting bigger in time, using platforms like Azure Cognitive Search or other VectorDBs would be a better option. Our next [notebook](../05_VectorDB/01_CognitiveSearch.ipynb) will show how to use Azure Cognitive Search to search for similar questions. 